In [1]:
# 필수 라이브러리 설치 (최초 1회만 실행)
import time
import pandas as pd
import numpy as np
from typing import List, Dict, Callable
from sentence_transformers import SentenceTransformer, util
import re

In [ ]:
# 1. 모델 코드 파일 Import
# 이 스크립트와 같은 경로에 각 모델의 .py 파일이 있어야 합니다.
# 실제 함수명과 다를 경우, 아래 부분을 수정해주세요.
try:
    from AX4 import get_ax4_response
    from exaone_32B import get_exaone_response
    from gemini_2_5_pro import get_gemini_response
    from llama_3_3_70b import get_llama_response
except ImportError as e:
    print(f"오류: 모델 코드를 불러오는 데 실패했습니다. 파일명을 확인해주세요. ({e})")

    # 실제 환경에서는 이 부분에서 종료하거나 다른 처리가 필요합니다.
    # 테스트를 위해 임시 함수를 정의합니다.
    def get_ax4_response(q):
        return "가상의 답변"

    def get_exaone_response(q):
        return "가상의 답변"

    def get_gemini_response(q):
        return "가상의 답변"

    def get_llama_response(q):
        return "가상의 답변"

c:\Users\user\Desktop\medical_chatbot\sj_model\model_test\AX4.py:32: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")


새로운 세션이 시작되었습니다. session_id: b514c673-ecc0-44ea-a861-c63c388de988



c:\Users\user\Desktop\medical_chatbot\sj_model\model_test\AX4.py:162: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



[AX-4.0 응답]
현재 입원 중이시라면, 긍정적인 취미 활동을 하거나 독서, 가벼운 운동, 혹은 다른 입원 환자분들과 소통하시며 시간을 보내는 것도 도움이 될 수 있습니다. 가능하다면 병원 자체에서 제공하는 프로그램에도 참여해보세요. 그리고 간병인이나 가족분들과 대화를 나눠 보는 것도 좋습니다.

[AX-4.0 응답]
즐거운 시간을 보내실 수 있는 다양한 방법을 추가로 제안해 드리겠습니다.

1. **독서**: 좋아하는 책이나 잡지를 읽어보세요.
2. **음악 감상**: 좋아하는 음악을 듣거나, 병원에서 제공하는 디지털 스트리밍 서비스를 이용해 보세요.
3. **영화 시청**: 병원 방 안에서 태블릿이나 노트북으로 영화를 보는 것도 좋은 방법입니다.
4. **가벼운 퍼즐**: 맞추기 쉬운 퍼즐이나 색칠놀이 책은 시간을 즐겁게 보낼 수 있습니다.
5. **일기 쓰기**: 현재의 감정이나 경험을 일기로 적어보세요.
6. **사람과 소통**: 간호사나 가족, 친구들과 이야기 나누며 서로의 일상을 공유하는 것도 도움이 됩니다.
7. **명상 및 심호흡**: 잠깐의 명상이나 심호흡으로 마음을 편안하게 하는 것도 좋습니다.
8. **온라인 강연 시청**: 관심 있는 주제로 온라인 강연이나 강좌를 들으며 유익한 시간을 보내보세요.

이러한 활동들이 조금이나마 병원생활의 무료함을 덜어주고 기분 전환에 도움이 되기를 바랍니다. 건강하고 안전한 입원이 되시길 기원합니다.
프로그램을 종료합니다.
오류: 모델 코드를 불러오는 데 실패했습니다. 파일명을 확인해주세요. (cannot import name 'get_ax4_response' from 'AX4' (c:\Users\user\Desktop\medical_chatbot\sj_model\model_test\AX4.py))


In [ ]:
# 2. RAG 지표 측정을 위한 환경 설정
# 이 부분은 실제 RAG 시스템의 '검색' 단계를 시뮬레이션
# 실제 환경에서는 RAG 파이프라인의 검색 모듈을 직접 사용

# 문서 말뭉치 (Corpus) - 테스트용 가상 문서들
CORPUS = {
    "doc1": "건선은 피부에 붉은 반점과 비늘이 생기는 만성 염증성 피부 질환입니다. 면역 체계의 이상 반응으로 발생합니다.",
    "doc2": "입원 생활이 지루할 때는 독서, 음악 감상, 영화 시청, 친구들과의 화상 통화 등으로 시간을 보낼 수 있습니다. 회복에 집중하는 것도 중요합니다.",
    "doc3": "피부 증상이 지속되면 피부과 전문의와 상담하는 것이 가장 좋습니다. 정확한 진단과 치료 계획을 세우는 데 필수적입니다.",
    "doc4": "입원 기간 중에는 회복을 위해 충분한 휴식을 취하고, 건강한 식단을 유지하는 것이 좋습니다.",
    "doc5": "건선은 전염되지 않으며, 꾸준한 관리와 치료로 증상을 완화시킬 수 있습니다.",
}

# 유사도 계산을 위한 모델 (다국어 모델 사용)
model_for_embedding = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# 문서 임베딩 벡터 미리 생성
corpus_embeddings = model_for_embedding.encode(
    list(CORPUS.values()), convert_to_tensor=True
)

c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
# 3. 테스트 질문 및 정답(Ground Truth) 정의
# 유사도, Recall@k, MRR 측정을 위해 '정답 문서 ID'를 미리 정의
TEST_DATA = [
    {
        "type": "의료 질문",
        "question": "살이 자꾸 마르고 간지러워서 긁었더니 딱딱거리고 따끔거리는데 괜찮은거겠지?",
        "correct_doc_ids": ["doc1", "doc3", "doc5"],  # 이 질문에 대한 정답 문서
        "ground_truth_answer_keywords": [
            "건선",
            "피부과",
            "상담",
        ],  # 정확도 측정용 키워드
    },
    {
        "type": "일상 질문",
        "question": "나 입원했는데 심심해",
        "correct_doc_ids": ["doc2", "doc4"],  # 이 질문에 대한 정답 문서
        "ground_truth_answer_keywords": [
            "입원",
            "심심",
            "휴식",
            "독서",
        ],  # 정확도 측정용 키워드
    },
]

In [ ]:
# 4. RAG 지표 계산 함수
def evaluate_retrieval(query: str, test_data: Dict, k: int = 5) -> Dict:
    """
    쿼리와 테스트 데이터를 바탕으로 문서 검색 성능 지표를 계산합니다.
    """
    query_embedding = model_for_embedding.encode(query, convert_to_tensor=True)

    # 코사인 유사도를 사용한 검색
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=k)[0]
    retrieved_doc_ids = [list(CORPUS.keys())[hit["corpus_id"]] for hit in hits]

    # 유사도 점수: 가장 높은 유사도 점수
    max_similarity = hits[0]["score"] if hits else 0.0

    # Recall@k: 정답 문서가 상위 k개에 포함되었는지 여부
    is_recall_k = any(
        doc_id in retrieved_doc_ids for doc_id in test_data["correct_doc_ids"]
    )

    # MRR (Mean Reciprocal Rank): 첫 번째 정답 문서의 순위 역수
    mrr_score = 0.0
    for rank, doc_id in enumerate(retrieved_doc_ids, start=1):
        if doc_id in test_data["correct_doc_ids"]:
            mrr_score = 1.0 / rank
            break

    return {
        "retrieved_doc_ids": retrieved_doc_ids,
        "max_similarity": max_similarity,
        "is_recall_k": is_recall_k,
        "mrr_score": mrr_score,
    }

In [ ]:
# 5. 전체 평가 실행 및 결과 요약
def run_full_evaluation(models_to_evaluate: Dict[str, Callable]) -> pd.DataFrame:
    """
    모든 모델과 모든 테스트 데이터를 대상으로 5가지 지표를 계산합니다.
    """
    final_results = []

    for model_name, response_func in models_to_evaluate.items():
        for test_data in TEST_DATA:
            question = test_data["question"]

            # 1. 응답 속도(Latency) 측정
            start_time = time.time()
            try:
                response = response_func(question)
            except Exception as e:
                response = f"API 호출 실패: {e}"
                print(f"  > '{model_name}' - '{question}'에 대한 API 호출 실패.")
            end_time = time.perf_counter()
            latency = end_time - start_time
            print(f"응답 속도: {latency:.4f}초")

            # 2. 정확도(Accuracy) 측정 (키워드 기반)
            is_accurate = all(
                keyword in response
                for keyword in test_data["ground_truth_answer_keywords"]
            )

            # 3. Retrieval 지표 계산 (유사도, Recall@k, MRR)
            # test_data를 인자로 전달하여 함수 내에서 접근 가능하도록 수정
            retrieval_metrics = evaluate_retrieval(question, test_data)

            final_results.append(
                {
                    "모델명": model_name,
                    "질문 유형": test_data["type"],
                    "응답 속도 (초)": latency,
                    "정확도": is_accurate,
                    "유사도 점수": retrieval_metrics["max_similarity"],
                    "Recall@k (k=5)": retrieval_metrics["is_recall_k"],
                    "MRR": retrieval_metrics["mrr_score"],
                }
            )

    return pd.DataFrame(final_results)


if __name__ == "__main__":
    # 각 모델의 호출 함수를 딕셔너리에 매핑
    models_to_evaluate = {
        "sk/A.X4.0": get_ax4_response,
        "LGAI-EXAONE/EXAONE-4.0-32B": get_exaone_response,
        "google/gemini-2.5-pro": get_gemini_response,
        "llama-3.3-70b": get_llama_response,
    }

    print("=============== RAG 모델 성능 평가 시작 ===============")

    # 평가 실행
    df_results = run_full_evaluation(models_to_evaluate)

    print("\n\n=============== 최종 평가 결과 ================")
    # 결과를 보기 좋은 표로 출력
    print(df_results)

    # 평균 결과 요약
    print("\n\n=============== 모델별 평균 성능 요약 ================")
    df_summary = (
        df_results.groupby("모델명")
        .agg(
            {
                "응답 속도 (초)": "mean",
                "정확도": lambda x: (x == True).sum() / len(x),
                "유사도 점수": "mean",
                "Recall@k (k=5)": "mean",
                "MRR": "mean",
            }
        )
        .reset_index()
    )

    # 요약 표 출력
    print(df_summary)

    # 최종 결론
    print("\n\n=============== 최종 분석 ================")
    print(
        "이 표를 참고하여 각 모델의 강점과 약점을 파악하고, 프로젝트의 목표에 가장 적합한 모델을 선택할 수 있습니다."
    )

=============== RAG 모델 성능 평가 시작 ===============
응답 속도: -1754696443.1533초
응답 속도: -1754696443.1539초
응답 속도: -1754696443.1543초
응답 속도: -1754696443.1541초


c:\Users\user\Desktop\medical_chatbot\.venv\lib\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


응답 속도: -1754696443.1541초
응답 속도: -1754696443.1534초
응답 속도: -1754696443.1537초
응답 속도: -1754696443.1539초


=============== 최종 평가 결과 ================
                          모델명  질문 유형     응답 속도 (초)    정확도    유사도 점수  \
0                   sk/A.X4.0  의료 질문 -1.754696e+09  False  0.510106   
1                   sk/A.X4.0  일상 질문 -1.754696e+09  False  0.369330   
2  LGAI-EXAONE/EXAONE-4.0-32B  의료 질문 -1.754696e+09  False  0.510106   
3  LGAI-EXAONE/EXAONE-4.0-32B  일상 질문 -1.754696e+09  False  0.369330   
4       google/gemini-2.5-pro  의료 질문 -1.754696e+09  False  0.510106   
5       google/gemini-2.5-pro  일상 질문 -1.754696e+09  False  0.369330   
6               llama-3.3-70b  의료 질문 -1.754696e+09  False  0.510106   
7               llama-3.3-70b  일상 질문 -1.754696e+09  False  0.369330   

   Recall@k (k=5)  MRR  
0            True  0.5  
1            True  1.0  
2            True  0.5  
3            True  1.0  
4            True  0.5  
5            True  1.0  
6            True  0.5  
7            Tru